# German Vocabulary Anki Card Enricher

This notebook creates enriched German vocabulary cards for Anki with example sentences and audio pronunciations.

## ✨ Key Features:
- **Smart input**: Provide either German or English words - the system automatically translates and generates complete cards
- **Grammatical information**: Automatically includes articles (der/die/das) and plurals for nouns, verb conjugations for verbs
- **Multiple examples**: 3 example sentences with translations for each word
- **Individual audio**: Separate audio file for each example sentence (no duplicate audio)
- **Professional cards**: Clean, well-formatted Anki cards ready for studying

## 🎯 How to Use:
1. Create a `.env` file with your Anthropic API key (see instructions below)
2. Run all cells to set up the environment
3. Call `create_card("word", "language")` with any German or English word
4. Export your deck and import into Anki!

## Setup and Dependencies

First, let's install the required packages if you don't have them already:

### Using .env Files for API Keys

To keep your API keys secure and out of your code, create a `.env` file in your project directory with the following content:

```
ANTHROPIC_API_KEY=your-actual-api-key-here
```

Then add `.env` to your `.gitignore` file to avoid committing your keys to version control.

In [88]:
%pip install genanki gtts requests ipython openai anthropic python-dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [89]:
import os
from dotenv import load_dotenv

load_dotenv()

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

## Import Libraries

In [90]:
import json
import random
import genanki
import gtts
from IPython.display import Audio, display

## Define Anki Model and Deck

In [91]:
# Define the Anki note model
model_id = random.randrange(1 << 30, 1 << 31)
german_model = genanki.Model(
    model_id,
    'German Vocabulary Model',
    fields=[
        {'name': 'German'},
        {'name': 'English'},
        {'name': 'Example'},
        {'name': 'Audio'},
        {'name': 'Notes'}
    ],
    templates=[
        {
            'name': 'German -> English',
            'qfmt': '<div class="german">{{German}}</div>',
            'afmt': '{{FrontSide}}<hr id="answer"><div class="english">{{English}}</div><br><div class="notes">{{Notes}}</div><br><div class="examples">{{Example}}</div><br>{{Audio}}',
        },
        {
            'name': 'English -> German',
            'qfmt': '<div class="english">{{English}}</div>',
            'afmt': '{{FrontSide}}<hr id="answer"><div class="german">{{German}}</div><br><div class="notes">{{Notes}}</div><br><div class="examples">{{Example}}</div><br>{{Audio}}',
        }
    ],
    css='''.card { 
        font-family: arial; 
        font-size: 20px; 
        text-align: center; 
        color: black; 
        background-color: white; 
    }
    .german { 
        font-weight: bold; 
        color: #3399ff; 
    }
    .english { 
        font-style: italic; 
    }
    .examples { 
        text-align: left;
        margin: 15px;
    }'''
)

# Create a deck
deck_id = random.randrange(1 << 30, 1 << 31)
deck = genanki.Deck(deck_id, 'German Vocabulary')

## Functions for Fetching Examples and Generating Audio

In [92]:
import anthropic

def get_word_info_and_examples(word, language, max_sentences=3):
    """Get complete word information including translation, grammar, and examples
    
    Args:
        word: The word to process
        language: 'german' or 'english' to specify the input language
        max_sentences: Number of example sentences to generate
    """
    
    client = anthropic.Anthropic(api_key=anthropic_api_key)
    
    if language == 'german':
        prompt = f"""Analyze the German word '{word}' and provide:
1. English translation
2. Word type (noun, verb, adjective, etc.)
3. If noun: definite article (der/die/das) and plural form
4. If verb: present tense (ich/du/er forms), past tense (ich form), and perfect tense (ich form)
5. {max_sentences} B1-level example sentences in German with English translations

Return as JSON with this structure:
{{
  "german": "correct German spelling",
  "english": "English translation", 
  "word_type": "noun|verb|adjective|etc",
  "grammar": {{
    "article": "der|die|das (for nouns only)",
    "plural": "plural form (for nouns only)",
    "present": "ich forme, du formst, er formt (for verbs only)",
    "past": "ich formte (for verbs only)", 
    "perfect": "ich habe geformt (for verbs only)"
  }},
  "sentences": [{{"example": "German text", "translation": "English text"}}]
}}"""
    else:
        prompt = f"""Translate the English word '{word}' to German and provide:
1. German translation
2. Word type (noun, verb, adjective, etc.)
3. If noun: definite article (der/die/das) and plural form  
4. If verb: present tense (ich/du/er forms), past tense (ich form), and perfect tense (ich form)
5. {max_sentences} B1-level example sentences in German with English translations

Return as JSON with this structure:
{{
  "german": "German translation",
  "english": "original English word",
  "word_type": "noun|verb|adjective|etc", 
  "grammar": {{
    "article": "der|die|das (for nouns only)",
    "plural": "plural form (for nouns only)",
    "present": "ich forme, du formst, er formt (for verbs only)",
    "past": "ich formte (for verbs only)",
    "perfect": "ich habe geformt (for verbs only)"
  }},
  "sentences": [{{"example": "German text", "translation": "English text"}}]
}}"""
    
    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1500,
        temperature=0.7,
        system="""You are a helpful German language assistant. You respond only in valid JSON without any additional formatting. Provide accurate grammatical information and natural example sentences.""",
        messages=[{"role": "user", "content": prompt}]
    )
    
    result = json.loads(response.content[0].text)
    return result

In [93]:
# Function to generate audio for a word or sentence
def generate_audio(text, filename):
    """Generate audio for German text using gTTS"""
    tts = gtts.gTTS(text, lang='de')
    tts.save(filename)
    return filename

## Card Creation and Export Functions

In [94]:
def create_card(word, language):
    """
    Create an Anki card for a German vocabulary word using Anthropic API.
    Takes a word and its language ('german' or 'english') and generates the complete card with:
    - Translation
    - Grammatical information (articles, plurals, verb forms)
    - Example sentences with translations
    - Audio for each example sentence
    Returns list of audio files and word info for preview.
    Requires an API key - will raise an exception if the API call fails.
    """
    # Create audio directory if it doesn't exist
    if not os.path.exists('audio'):
        os.makedirs('audio')
    
    # Get complete word information from API
    word_info = get_word_info_and_examples(word, language)
    
    german = word_info['german']
    english = word_info['english']
    word_type = word_info['word_type']
    grammar = word_info['grammar']
    examples = word_info['sentences']
    
    # Build grammatical notes
    grammar_notes = []
    if word_type == 'noun' and grammar.get('article'):
        grammar_notes.append(f"{grammar['article']} {german}")
        if grammar.get('plural'):
            grammar_notes.append(f"Plural: {grammar['plural']}")
    elif word_type == 'verb':
        if grammar.get('present'):
            grammar_notes.append(f"Present: {grammar['present']}")
        if grammar.get('past'):
            grammar_notes.append(f"Past: {grammar['past']}")
        if grammar.get('perfect'):
            grammar_notes.append(f"Perfect: {grammar['perfect']}")
    
    # Combine all notes
    all_notes = []
    if grammar_notes:
        all_notes.extend(grammar_notes)
    notes_text = " | ".join(all_notes)
    
    # Format examples as paired sentences with translations (NO audio in examples)
    formatted_examples = []
    audio_files = []
    audio_tags = []
    
    for i, example in enumerate(examples, 1):
        # Format each example as: German sentence, English translation
        german_sentence = example['example']
        english_translation = example['translation']
        
        # Generate audio file for this sentence
        audio_filename = f"audio/{german.lower().replace(' ', '_')}_ex{i}.mp3"
        generate_audio(german_sentence, audio_filename)
        audio_files.append(audio_filename)
        
        # Create the formatted example WITHOUT audio tags
        formatted_example = f"""<b>{i}. {german_sentence}</b><br>
        <i>{english_translation}</i>"""
        
        formatted_examples.append(formatted_example)
        audio_tags.append(f'[sound:{os.path.basename(audio_filename)}]')
    
    # Join all formatted examples
    all_examples = "<br><br>".join(formatted_examples)
    
    # Create the note for Anki
    note = genanki.Note(
        model=german_model,
        fields=[
            german,
            english,
            all_examples,
            " ".join(audio_tags),  # All audio files in Audio field only
            notes_text
        ]
    )
    
    # Add the note to the deck
    deck.add_note(note)
    
    # Return all audio files and word info for preview
    return audio_files, word_info


In [95]:
# Function to export the deck
def export_deck(filename='german_vocabulary.apkg'):
    """Export the deck to an Anki package"""
    # Create media files collection
    media_files = []
    for file in os.listdir('audio'):
        if file.endswith('.mp3'):
            media_files.append(os.path.join('audio', file))
    
    # Create and save the package
    package = genanki.Package(deck)
    package.media_files = media_files
    package.write_to_file(filename)
    print(f"Deck exported as {filename}")

## Demo & Usage Examples

In [96]:
# Demo: Create a few sample cards
def demo():
    """Create sample cards for demonstration"""
    # Just provide the words - can be German or English!
    words = [
        ('Haus', "german"),      # German noun
        # ('butterfly', "english"), # English noun  
        # ('lernen', "german")     # German verb
    ]
    
    for (word, language) in words:
        try:
            audio_files, word_info = create_card(word, language)
            print(f"Created card: {word_info['german']} → {word_info['english']}")
            print(f"Word type: {word_info['word_type']}")
            if word_info['grammar']:
                print(f"Grammar info: {word_info['grammar']}")
            print("Examples:")
            for i, example in enumerate(word_info['sentences'], 1):
                print(f"  {i}. {example['example']} → {example['translation']}")
                # Play audio for this example
                if i <= len(audio_files):
                    display(Audio(audio_files[i-1]))
            print()
        except Exception as e:
            print(f"Failed to create card for '{word}': {e}")
            print()
    
    export_deck()

In [97]:
# Run the demo
demo()

Created card: Haus → house
Word type: noun
Grammar info: {'article': 'das', 'plural': 'Häuser', 'present': None, 'past': None, 'perfect': None}
Examples:
  1. Mein Haus liegt am Stadtrand von Berlin. → My house is located on the outskirts of Berlin.


  2. Wir haben letztes Jahr ein neues Haus gebaut. → We built a new house last year.


  3. In diesem Haus wohnen drei Familien zusammen. → Three families live together in this house.



Deck exported as german_vocabulary.apkg
